# 使用 OpenAI 接口完成主题建模和情感分析任务

In [ ]:
import os
from openai import OpenAI
from rich import print as pp

# 1. 准备环境

## 1.1. 初始化 OpenAI 客户端

In [2]:
# 尝试阅读环境变量
try:
    OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
    if OPENAI_API_KEY is None:
        raise KeyError
    pp("[green]环境变量 OPENAI_API_KEY 已成功读取。")
except KeyError:
    OPENAI_API_KEY = None
    pp("[red]未找到环境变量 OPENAI_API_KEY。请确保已正确设置。")

client = OpenAI(api_key=OPENAI_API_KEY)  # 替换为你的 API 密钥

环境变量 OPENAI_API_KEY 已成功读取。

# 1. 情绪分类


## 1.1. 定义方法

In [3]:
def analyze_sentiment(text: str) -> str:
    """
    根据给定文本使用 GPT-4 模型进行情感分析
    
    参数:
        text (str): 需要进行情感分析的文本
    
    返回:
        str: GPT-4 模型返回的情感分析结果
    """
    # 构建情感分析提示
    system_prompt = "你是一个专业的情感分析助手，能够准确识别和解释文本中的情感倾向。如果是积极的，则返回`1`，如果是消极的，则返回`-1`，如果是中性的，则返回`0`。如果无法判断，则给出最合适的猜测。请只返回结果。"
    user_prompt = f"请对下面的文本进行情感分析：\n\n\"{text}\"\n\n情感分析结果:"
    
    # 调用 GPT-4 模型
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    
    # 提取并返回结果
    return response.choices[0].message.content.strip()

result_map = {
    "1": "积极",
    "-1": "消极",
    "0": "中性"
}

## 1.2 运行

### 示例:积极

In [4]:
# 提供一个示例文本进行情感分析
sample_text = "我今天心情非常愉快，因为得到了一个意想不到的惊喜！"
# 调用情感分析函数
result = analyze_sentiment(sample_text)
# 输出结果
pp(f"[blue]文本:[/blue] {sample_text}\n[green]情感:[/green] {result_map.get(result, '未知')}")

文本: 我今天心情非常愉快，因为得到了一个意想不到的惊喜！
情感: 积极

### 示例:消极

In [5]:
# 提供一个示例文本进行情感分析
sample_text = "我今天非常倒霉,没抢到心仪的演唱会门票。"
# 调用情感分析函数
result = analyze_sentiment(sample_text)
# 输出结果
pp(f"[blue]文本:[/blue] {sample_text}\n[green]情感:[/green] {result_map.get(result, '未知')}")

文本: 我今天非常倒霉,没抢到心仪的演唱会门票。
情感: 消极

# 2. 主题建模


## 2.1. 定义方法

In [6]:
def topic_modeling(text: str) -> str:
    """
    根据给定文本使用 GPT-4 模型进行主题建模
    
    参数:
        text (str): 需要进行主题建模的文本
    
    返回:
        str: GPT-4 模型返回的主题列表，多个主题用逗号分隔
    """
    # 构建主题建模提示
    system_prompt = "你是一个专业的主题建模助手，能够准确识别文本中包含的主要主题。请分析文本并从['政治','娱乐']两个类别中返回最相关的一个主题。如果无法确定，请给出最合理的猜测。请只返回结果。"
    user_prompt = f"请对下面的文本进行主题建模：\n\n\"{text}\"\n\n主题:"
    
    # 调用 GPT-4 模型
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    
    # 提取并返回结果
    return response.choices[0].message.content.strip()


## 2.2. 运行

### 示例:政治

In [7]:
topic_modeling("中国国防部召开记者会")

'政治'

### 示例:娱乐

In [8]:
topic_modeling("霉霉在 TikTok 开设账号,吸引年轻人")

'娱乐'